# User flow

In [12]:
#File for goals (goals_dict)

request1 = "POST https://analyticsreporting.googleapis.com/v4/reports:batchGet?fields=reports(columnHeader%2Cdata(rows%2Ctotals))&key={YOUR_API_KEY}"
request1 = {
 "reportRequests": [
  {
   "viewId": "123303369",
   "dateRanges": [
    {
     "startDate": "2017-01-01",
     "endDate": "2017-04-30"
    }
   ],
   "metrics": [
    {
     "expression": "ga:goal1Completions" #instead of "ga:goal1Completions" use goal_to_use_in_request variable from tracking-tags code
    }
   ],
   "dimensions": [
    {
     "name": "ga:goalCompletionLocation"
    }
   ]
  }
 ]
}

In [13]:
#File for sessions (sessions_dict)

request2 = "POST https://analyticsreporting.googleapis.com/v4/reports:batchGet?fields=reports(columnHeader%2Cdata(rows%2Ctotals))&key={YOUR_API_KEY}"
 
request2 = {
 "reportRequests": [
  {
   "viewId": "123303369",
   "dateRanges": [
    {
     "startDate": "2017-01-01",
     "endDate": "2017-04-30"
    }
   ],
   "metrics": [
    {
     "expression": "ga:sessions"
    }
   ],
   "dimensions": [
    {
     "name": "ga:pagePath"
    }
   ]
  }
 ]
}

In [14]:
import json

with open('files/TMRW_goal1_goalloc.json') as file: 
    input_goals = json.load(file)

goals = input_goals["reports"][0]['data']['rows']
#goals

In [15]:
with open('files/TMRW_sess_page.json') as file: 
    input_sess = json.load(file)

sessions = input_sess["reports"][0]['data']['rows']
#sessions

In [16]:
def create_dict(x):
    goals_dict = {}

    for s in x:
        
        #print (str((s['dimensions'])))
        #print (s['metrics'][0]['values'])
        #session_dict[] = 0
        goals_dict[s['dimensions'][0]] = int(s['metrics'][0]['values'][0])
        #session_dict[s['dimensions']] = s['metrics'][0]['values']
    
    return goals_dict
        

goals_dict = (create_dict(goals))

goals_dict

{'(entrance)': 6,
 '/': 85,
 '/TMRW_Byte_Cafe.php': 22,
 '/TMRW_FAQs.php': 24,
 '/TMRW_Get_in_touch.php': 3,
 '/TMRW_the_team.php': 5,
 '/portfolio-single-gallery.html': 1}

In [17]:
# get totals

total_conv = int(input_goals['reports'][0]['data']['totals'][0]['values'][0])

total_sess = int(input_sess['reports'][0]['data']['totals'][0]['values'][0])

total_sess

6893

In [18]:
def create_dict(x):
    sessions_dict = {}

    for s in x:
        
        #print (str((s['dimensions'])))
        #print (s['metrics'][0]['values'])
        #session_dict[] = 0
        sessions_dict[s['dimensions'][0]] = int(s['metrics'][0]['values'][0])
        #session_dict[s['dimensions']] = s['metrics'][0]['values']
    
    return sessions_dict
        

sessions_dict = (create_dict(sessions))

#sessions_dict

In [19]:
# Filter page with less than 3% of conversions and less than 1% of sessions

# define thresholds for filter
threshold_conv = round(total_conv * 0.03,0)
threshold_sess = round(total_sess * 0.01,0)

def filter_data(data,thr):
    result = {}
    for page in data:
        if data[page] > thr:
            #print (data[page])
            result[page] = 0
            result[page] = data[page]
    
    return result

goals_dict = filter_data(goals_dict,threshold_conv)
sessions_dict = filter_data(sessions_dict,threshold_sess)
sessions_dict

{'/': 5925,
 '/TMRW_Byte_Cafe.php': 270,
 '/TMRW_FAQs.php': 129,
 '/TMRW_the_team.php': 140,
 '/trainstrikes.php': 231,
 '/voteforbyte.php': 79}

In [20]:
goals_dict_keys = set(goals_dict.keys())
goals_dict_keys

{'(entrance)',
 '/',
 '/TMRW_Byte_Cafe.php',
 '/TMRW_FAQs.php',
 '/TMRW_the_team.php'}

In [21]:
def dict_compare(goals_dict, sessions_dict):
    
    goals_dict_keys = set(goals_dict.keys())
    
    sessions_dict_keys = set(sessions_dict.keys())
    
    intersect_keys = goals_dict_keys.intersection(sessions_dict_keys)
    
    return intersect_keys


correct_pages = dict_compare(goals_dict, sessions_dict)
correct_pages

{'/', '/TMRW_Byte_Cafe.php', '/TMRW_FAQs.php', '/TMRW_the_team.php'}

In [22]:
conv_rate_dict = {}

for page in correct_pages:
    for page2 in goals_dict:
        if page == page2:
            #print(page2)
        
        
            conv_rate = (goals_dict[page]/(sessions_dict[page])*100)
            conv_rate_dict[page2] = conv_rate 
        

conv_rate_dict

{'/': 1.4345991561181435,
 '/TMRW_Byte_Cafe.php': 8.148148148148149,
 '/TMRW_FAQs.php': 18.6046511627907,
 '/TMRW_the_team.php': 3.571428571428571}

In [23]:
dictValues = []

# For each key in the dict's Values,
for x in conv_rate_dict.values():
    # add the key to dictValues
    dictValues.append(x)

# View the dictionaryValues list
dictValues

[18.6046511627907, 8.148148148148149, 3.571428571428571, 1.4345991561181435]

In [24]:
max_value = max(dictValues)
max_value = "{0:.2f}%".format(round(max_value,2))
max_value

'18.60%'

In [25]:
min_value = min(dictValues)
min_value = "{0:.2f}%".format(round(min_value,2))
min_value

'1.43%'

In [26]:
max_page = max(conv_rate_dict, key=conv_rate_dict.get)

if max_page == '/':
    max_page = "homepage"
else:
    max_page = max_page

max_page

'/TMRW_FAQs.php'

In [27]:
min_page = min(conv_rate_dict, key=conv_rate_dict.get)

if min_page == '/':
    min_page = "homepage"
else:
    min_page = min_page

min_page


'homepage'

### Print

In [28]:
print('Page \'%s\' has the highest Conversion Rate - %s.' % (max_page, max_value))

print('Page \'%s\' has the lowest Conversion Rate - %s.' % (min_page.title(), min_value))

Page '/TMRW_FAQs.php' has the highest Conversion Rate - 18.60%.
Page 'Homepage' has the lowest Conversion Rate - 1.43%.


In [29]:
g = {'/': 1.4345991561181435,
 '/TMRW_Byte_Cafe.php': 8.148148148148149,
 '/TMRW_FAQs.php': 18.6046511627907,
 '/TMRW_the_team.php': 3.571428571428571}

### Visualisation

In [190]:
import plotly

import plotly.plotly as py
import plotly.graph_objs as go

py.sign_in('m-nudha', 'D8jC4ovl5yHKcvRy0nWk')

In [192]:
trace0 = go.Scatter(
    
    x = [5925, 270, 129, 140, 231, 79],  #[85.96, 0.039, 0.019, 0.02, 3.35, 1.146],
    #[85, 24, 22, 5, 1, 1], 
    y = [1.4345991561181435, 8.148148148148149, 18.6046511627907, 3.571428571428571, 0, 0],
    text = ['/', '/TMRW_Byte_Cafe.php', '/TMRW_FAQs.php', '/TMRW_the_team.php','/trainstrikes.php','/voteforbyte.php'],
    mode = 'markers',
    marker = dict(
        color = ['rgb(93, 164, 214)', 'rgb(255, 144, 14)',
                 'rgb(44, 160, 101)', 'rgb(255, 65, 54)',
                 'rgb(44, 160, 101)', 'rgb(255, 144, 14)'],
        opacity=[1, 0.9, 0.8, 0.7, 0.6, 0.5],
        size= [85, 24, 22, 5, 1, 1], #[85.96, 0.039, 0.019, 0.02, 3.35, 1.146],  # [5925, 270, 129, 140, 231, 79], 
        
    )
)

data = [trace0]


layout = go.Layout(
    title='Conversions vs Sessions',
    xaxis=dict(
        title='Conversions',
        gridcolor='rgb(255, 255, 255)',
        range=[-10, 500],
        type='log',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Conversion Rate',
        gridcolor='rgb(255, 255, 255)',
        range=[-5, 25],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='life-expectancy-per-GDP-2007')
py.iplot(data, filename='bubblechart-color')

PlotlyRequestError: Hey there! You've hit one of our API request limits. 

To get unlimited API calls(10,000/day), please upgrade to a paid plan. 

UPGRADE HERE: https://goo.gl/i7glmM 

Thanks for using Plotly! Happy Plotting!